In [1]:
from nltk.corpus import wordnet as wn
import pandas as pd
import numpy as np
#from utils.polyhierarchy_comparison_measures import SP_weighted_embeddings_wordnet,SP_max_embeddings_wordnet
import itertools
import logging
import operator
import numpy as np
from scipy import spatial
import networkx as nx
import glob
from scipy import stats
from numpy.random import randn
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
wn_lemma = WordNetLemmatizer()
from scipy import spatial

In [2]:
def set_diag(self, values): 
    n = min(len(self.index), len(self.columns))
    self.values[tuple([np.arange(n)] * 2)] = values




In [3]:
import numpy as np
import matplotlib.pyplot as plt; plt.ion()
import networkx as nx

def find_lowest_common_ancestor(graph, a, b):
    """
    Find the lowest common ancestor in the directed, acyclic graph of node a and b.
    The LCA is defined as on

    @reference:
    https://en.wikipedia.org/wiki/Lowest_common_ancestor

    Notes:
    ------
    This definition is the opposite of the term as it is used e.g. in biology!

    Arguments:
    ----------
        graph: networkx.DiGraph instance
            directed, acyclic, graph

        a, b:
            node IDs

    Returns:
    --------
        lca: [node 1, ..., node n]
            list of lowest common ancestor nodes (can be more than one)
    """

    assert nx.is_directed_acyclic_graph(graph), "Graph has to be acyclic and directed."

    # get ancestors of both (intersection)
    common_ancestors = list(nx.ancestors(graph, a) & nx.ancestors(graph, b))
    
    # get sum of path lengths
    sum_of_path_lengths = np.zeros((len(common_ancestors)))
    for ii, c in enumerate(common_ancestors):
        sum_of_path_lengths[ii] = nx.shortest_path_length(graph, c, a) \
                                  + nx.shortest_path_length(graph,  c,b)

    # print common_ancestors
    # print sum_of_path_lengths
    
    # return minima
    minima, = np.where(sum_of_path_lengths == np.min(sum_of_path_lengths))
    #print(minima)
    return min(sum_of_path_lengths)
    #return [common_ancestors[ii] for ii in minima]

In [7]:
# create dictionary for the input words and 3 related synsets in the wordnet
wn_lemma = WordNetLemmatizer()
def word_synset3_dict(concept_names, word_type='noun'):
    leaf_nodes = dict()
    for c in concept_names:
        
        ls= []
        
        word1 = wn_lemma.lemmatize(c)
        
        if word_type=='verb':
            count = 0
            for result in wn.synsets(word1, pos = wn.VERB): # retrieve all concepts related to this word
                if count<10:
                    ls.append(result.name())
                    count+=1
                if ls:
                    leaf_nodes[word1] = ls
        if word_type=='noun':
            count = 0
            for result in wn.synsets(word1, pos = wn.NOUN): # retrieve all concepts related to this word

                # only add corresponding noun synsets '.n.' in result.name() and 
                if count<10:
                    ls.append(result.name())
                    count+=1
                if ls:
                    leaf_nodes[word1] = ls

        
    return leaf_nodes



In [8]:
# create dictionary for the input words and 3 related synsets in the wordnet

def word_synsetall_dict(concept_names, pos):
    leaf_nodes = dict()
    for c in concept_names:
        ls= []
        count = 0
        word1 = wn_lemma.lemmatize(c)
        for result in wn.synsets(word1): # retrieve all concepts related to this word
                ls.append(result.name())
        if ls:
            leaf_nodes[word1] = ls
    return leaf_nodes




In [9]:
# CREATE embeddings based on common ancestors between two nodes.
def SP_embeddings_wordnet(filename, target_filename,rootnode, word_pairs, lambda_factor=0.7, concept_dict=None):
    df = pd.read_csv(filename)
    # Create the Directed Graph 
    try:
        G = nx.from_pandas_edgelist(df,
                            source='parent',
                            target='child',
                            create_using=nx.DiGraph())
        # find level of node(shortest path from root to current node)
        optional_attrs = nx.shortest_path_length(G ,rootnode)
        nx.set_node_attributes(G ,  optional_attrs, 'node_level' )
        
        #word_pairs = list(itertools.product(list(concept_dict.keys()), repeat=2)) # create pair of all nodes 
       
        ancestors_distance = {}
        for word in word_pairs:        
            if word[0]== word[1]:
                ancestors_distance[word]=1
            else:
                common_ancestor = {} 
                # fetch synsets from the graph for both words
                if concept_dict:
                    concepts_word1 = concept_dict[word[0]]
                    concepts_word2 = concept_dict[word[1]]
                else:
                    print (word)
                    break
                pairs = list(itertools.product(concepts_word1, concepts_word2)) # create pair of all nodes 
                
                ls_similarity = []
                
                distance_list=[]
                distance_dict= {}
                for i in pairs:
                    minima = find_lowest_common_ancestor (G, i[0], i[1])
                    sim_m = lambda_factor**(( minima)/2) # divide may lead to float lamads , 2, 2,5                 
                    #distance_dict[i] = np.max(ls_distance)
                    distance_list.append (sim_m) # choose the distance for lowest common ancestor for given concept pair

                
                ancestors_distance[word] =   np.max(distance_list)


                
        chunked_data = [[k[0],k[1], v] for k, v in ancestors_distance.items()]
        df_nodes = pd.DataFrame(chunked_data)
        df_nodes = df_nodes.rename(columns= {0:'node1', 1:'node2', 2:'weight'})
        
         # create adjancey matrix
        vals = np.unique(df_nodes[['node1', 'node2']])
        df_nodes = df_nodes.pivot(index='node1', columns='node2', values='weight'
                          ).reindex(columns=vals, index=vals, fill_value=0)

        df_adjacency = df_nodes#.apply( lambda x:   lambda_factor** x)

        #set diagnoal to 1
        pd.DataFrame.set_diag = set_diag
        df_adjacency.set_diag(1)
        df_adjacency.fillna(0, inplace=True)


        df_adjacency.to_csv(target_filename)


        
        # create adjancey matrix
        df_adjacency.to_csv(target_filename)
        
        
    except BaseException:
        logging.exception("An exception was thrown!")

In [ ]:
graph_file = './Graph/graph_wordsimilarity.csv'
#graph_file = './Graph/Noungraph_wordsimilarity.csv'
data_files = glob.glob("datasets/*.csv")
#data_files =["datasets/NOUN_mc-30.csv"]
lambda_factor= 0.65#0.755 #0.9
strategy = 'SP'
column_names = ['word1', 'word2', 'sim']
root_node = 'root'
count =0
for file in data_files:
    if count==10:
        break
    else:
        count +=1
        targetfile = file.split('/')[1]
        word_list = []
        df = pd.read_csv(file) #,  names = column_names)#sep=';',
        
        word_list.append(df['word1'])
        word_list.append(df['word2'])  
        word_list = list(set([item for sublist in word_list for item in sublist]))
        
        
        # create dictionary of word synsets from given word list
        if file.split('/')[1].split('_')[0].lower() =='noun':
            print (file.split('/')[1])
            leaf_nodes = word_synset3_dict(word_list, 'noun')

            wd1 = [wn_lemma.lemmatize(w) for w in df['word1']]
            wd2 = [wn_lemma.lemmatize(w) for w in df['word2']]
            
            #word_pairs =list(zip(wd1, wd2))
            word_pairs = list(itertools.product(wd1+wd2, repeat=2))
           
            
            ls = leaf_nodes.keys() 
            # call all functions for each dataset
            
            if leaf_nodes :
                SP_embeddings_wordnet(graph_file,  './Embeddings/SP/'+targetfile, root_node ,word_pairs,lambda_factor, leaf_nodes)
            else:
                print ('No data found')


        elif file.split('/')[1].split('_')[0].lower() =='verb':
            print (file.split('/')[1])
            leaf_nodes = word_synset3_dict(word_list, 'verb')
            #
            wd1 = [wn_lemma.lemmatize(w) for w in df['word1']]
            wd2 = [wn_lemma.lemmatize(w) for w in df['word2']]
            
            #word_pairs =list(zip(wd1, wd2))
            word_pairs = list(itertools.product(wd1+wd2, repeat=2))
            ls = leaf_nodes.keys() 
           
            # call all functions for each dataset
            if leaf_nodes :
                SP_embeddings_wordnet(graph_file,  './Embeddings/SP/'+targetfile, root_node ,word_pairs,lambda_factor, leaf_nodes)
            else:
                pass
                #print ('No data found')
        else:
            #leaf_nodes = None
            pass
        
    
        

NOUN_rg-65.csv


ERROR:root:An exception was thrown!
Traceback (most recent call last):
  File "<ipython-input-9-f74070129093>", line 36, in SP_embeddings_wordnet
    minima = find_lowest_common_ancestor (G, i[0], i[1])
  File "<ipython-input-3-e44793908302>", line 31, in find_lowest_common_ancestor
    assert nx.is_directed_acyclic_graph(graph), "Graph has to be acyclic and directed."
  File "/opt/anaconda3/lib/python3.7/site-packages/networkx/algorithms/dag.py", line 107, in is_directed_acyclic_graph
    return G.is_directed() and not has_cycle(G)
  File "/opt/anaconda3/lib/python3.7/site-packages/networkx/algorithms/dag.py", line 87, in has_cycle
    consume(topological_sort(G))
  File "/opt/anaconda3/lib/python3.7/site-packages/networkx/utils/misc.py", line 240, in consume
    deque(iterator, maxlen=0)
  File "/opt/anaconda3/lib/python3.7/site-packages/networkx/algorithms/dag.py", line 178, in topological_sort
    zero_indegree = [v for v, d in G.in_degree() if d == 0]
  File "/opt/anaconda3/lib/py

VERB_verb-143.csv


ERROR:root:An exception was thrown!
Traceback (most recent call last):
  File "<ipython-input-9-f74070129093>", line 36, in SP_embeddings_wordnet
    minima = find_lowest_common_ancestor (G, i[0], i[1])
  File "<ipython-input-3-e44793908302>", line 31, in find_lowest_common_ancestor
    assert nx.is_directed_acyclic_graph(graph), "Graph has to be acyclic and directed."
  File "/opt/anaconda3/lib/python3.7/site-packages/networkx/algorithms/dag.py", line 107, in is_directed_acyclic_graph
    return G.is_directed() and not has_cycle(G)
  File "/opt/anaconda3/lib/python3.7/site-packages/networkx/algorithms/dag.py", line 87, in has_cycle
    consume(topological_sort(G))
  File "/opt/anaconda3/lib/python3.7/site-packages/networkx/utils/misc.py", line 240, in consume
    deque(iterator, maxlen=0)
  File "/opt/anaconda3/lib/python3.7/site-packages/networkx/algorithms/dag.py", line 184, in topological_sort
    for _, child in G.edges(node):
  File "/opt/anaconda3/lib/python3.7/site-packages/net

VERB_yp-130.csv


In [14]:
pearson_ls = []
spearman_ls = [] 
result_dict = {}
goldDatapath= './datasets/'
sim_data = glob.glob("./sim_scores/SP/*.csv")
column_names = ['word1', 'word2', 'sim']
algos = [  'lch']
columns =['SimLex222','Agirre201', 'Gerz', 'Millers', 'Rel122', 'wordsim353', 'yang' ]
df_score =pd.DataFrame(columns=columns)
index_ls = []
for file in sim_data:
    print (file)
    algoname = file.split('/')[2].split('sss')[0]
    
    filename = file.split('/')[-1]
    # read human similarity score
    df = pd.read_csv(goldDatapath+filename)
    score_human = df['similarity'].values
    norm1 = [(float(i)-min(score_human))/(max(score_human )-min(score_human )) for i in score_human ] # normalize score in 0-1
    norm1 = np.round(norm1,2)
    print ()
    for algoname in algos:
        # read similarity file
        df_r = pd.read_csv(file)
        
        algo_score = np.round( df_r['weight'].values,2)
        # normalize score
        #if (df_r['weight'] > 1).any():
        #algo_score= [(float(i)-min(algo_score))/(max(algo_score )-min(algo_score )) for i in algo_score ] # normalize score in 0-1
    
        # calculate Pearson's correlation
        corrp, _ = pearsonr(norm1, algo_score)
        pearson_ls.append(corrp) 
        print (np.round(corrp,2))
        corrs, _ = spearmanr(norm1, algo_score)
        #print('Spearmans correlation: %.3f' % corr)
        spearman_ls.append(corrs) 
#df_score.loc[len(df_score.index)+1] =pearson_ls
    

./sim_scores/SP/NOUN_rg-65.csv

0.7
./sim_scores/SP/VERB_verb-143.csv

0.56
./sim_scores/SP/VERB_yp-130.csv

0.54
./sim_scores/SP/NOUN_mc-30.csv

0.6


In [31]:
score_human = []
algo_score = []
for row in df.iterrows():
    w1 = row[1][0]
    w2 = row[1][1]
    sim = row[1][2]
    v1 = df_r.loc[df_r.node1 ==w1].values[0][1:]
    v2 = df_r.loc[df_r.node1 ==w2].values[0][1:]
    result = 1- spatial.distance.cosine(v1.astype('float64').reshape(-1, 1), v2.astype('float64').reshape(-1, 1))
    algo_score.append(result)
    score_human.append(sim)
    
norm1 = [(float(i)-min(score_human))/(max(score_human )-min(score_human )) for i in score_human ] # normalize score in 0-1
    # calculate Pearson's correlation
corr, _ = pearsonr(norm1, algo_score)
print (corr)
 

ValueError: could not convert string to float: 'brag'

In [42]:
v1 = df_r.loc[df_r.node1 =='boy'].values[0][1:]
v2 = df_r.loc[df_r.node1 =='lad'].values[0][1:]

In [43]:
v1

array([0.0, 0.0, 0.0, 1.0, 0.0, 0.5253293217295223, 0.0,
       0.6314054347710605, 0.0, 0.5253293217295223, 0.0,
       0.6314054347710605, 0.575930368, 0.0, 0.0, 0.0,
       0.43707399567896255, 0.6314054347710605, 0.6314054347710605,
       0.9121403400793104, 0.27597089626879995, 1.0, 0.43707399567896255,
       1.0, 0.0, 0.9121403400793104, 0.27597089626879995,
       0.8320000000000001, 0.4791740661759999, 0.6314054347710605, 1.0,
       0.3636455644048968, 0.5253293217295223, 0.575930368, 1.0,
       0.25172418717461525, 1.0], dtype=object)

In [44]:

result = 1- spatial.distance.cosine(v1.astype('float64').reshape(-1, 1), v2.astype('float64').reshape(-1, 1))
result

0.960144777628192

### Check results 
   
  

In [137]:
pearson_ls = []
spearman_ls = [] 
result_dict = {}
goldDatapath= './datasets/'
sim_data = glob.glob("./sim_scores/SP/*.csv")
column_names = ['word1', 'word2', 'sim']
algos = [  'lch']
columns =['SimLex222','Agirre201', 'Gerz', 'Millers', 'Rel122', 'wordsim353', 'yang' ]
df_score =pd.DataFrame(columns=columns)
index_ls = []
for file in sim_data:
    
    algoname = 'SP'
    index_ls.append(algoname+'sp')
    index_ls.append(algoname+'pr')
    filename = file.split('/')[3]
    print (filename)
    # read human similarity score
    df = pd.read_csv(goldDatapath+filename, sep=';', names = column_names)
    
    for algoname in algos:
        # read similarity file
        df_r = pd.read_csv(file)
        df = df.loc[df['word2'].isin( df_r['node2'].values)]
        #print (set(df['word2'].values)- set(df_r['node2'].values))
        score_human = df['sim'].values
        norm1 = [(float(i)-min(score_human))/(max(score_human )-min(score_human )) for i in score_human ] # normalize score in 0-1
    
        algo_score = df_r['sim'].values
        # normalize score
        if (df_r['sim'] > 1).any():
            algo_score= [(float(i)-min(algo_score))/(max(algo_score )-min(algo_score )) for i in algo_score ] # normalize score in 0-1
    
        print (len(norm1), len(algo_score))
        # calculate Pearson's correlation
        corrp, _ = pearsonr(norm1, algo_score)
        print (corrp)
        pearson_ls.append(corrp) 
        
        corrs, _ = spearmanr(norm1, algo_score)
        #print('Spearmans correlation: %.3f' % corr)
        spearman_ls.append(corrs) 

#df_score.loc[len(df_score.index)+1] =pearson_ls
    

Noun_Miller_Charles_28_dataset.csv
28 28
0.5407645615273217
Noun_WordSim353Full_dataset.csv
342 342
0.2785983495128621
Noun_Agirre201_lowercase_dataset.csv
201 201
0.5318889580062942
Verb_Yang_YP130_dataset.csv
126 126
0.5326846408687362
Verb_Gerz_SimVerb3500_dataset.csv
3488 3488
0.2365706753452848
Verb_SimLex222_verbs_dataset.csv
218 218
0.05083884005538274


In [133]:
df_score

,SimLex222,Agirre201,Gerz,Millers,Rel122,wordsim353,yang


In [47]:
data_files = glob.glob("./sim_scores/SPAllSynsets/*.csv")
pearson_ls = []
spearman_ls = []
columns = [col.split('/')[-1].split('.')[0] for col in data_files  ]
for file in data_files:
    pdf = pd.read_csv(file)
    print (pdf.shape)
    # read human similarity score       
    column_names = ['node1', 'node2', 'simHuman']
    word_list = []
    df = pd.read_csv('./datasets_wordpairSim/'+file.split('/')[-1].split('_')[-1], sep=';', names = column_names)
    
    r = pd.merge(pdf, df, on=['node1', 'node2'])
    score_human = r['simHuman']
    algo_score = r['sim']
    
    norm1 = [(float(i)-min(score_human))/(max(score_human )-min(score_human )) for i in score_human ] # normalize score in 0-1
    # calculate Pearson's correlation
    corr, _ = pearsonr(norm1, algo_score)
    pearson_ls.append(corr) 
 
    # calculate spearman's correlation
    corr, _ = spearmanr(norm1, algo_score)
    spearman_ls.append(corr) 
result_dict['SP : spearman'] = spearman_ls
result_dict['SP : Pearson'] = pearson_ls

denom =np.add (spearman_ls , pearson_ls)
ls = np.multiply(spearman_ls ,pearson_ls )
ls = [x * 2 for x in ls]
ls = ls/denom
#result_dict['SP : harmonic mean'] =  ls

(30, 4)
(347, 4)
(144, 4)
(56, 4)
(130, 4)
(408, 4)
(1989, 4)
(203, 4)


In [49]:
data_files = glob.glob("./sim_scores/LeacockAllSynsets/*.csv")
pearson_ls = []
spearman_ls = []

for file in data_files:
    pdf = pd.read_csv(file)
    
    # read human similarity score       
    column_names = ['node1', 'node2', 'simHuman']
    word_list = []
    df = pd.read_csv('./datasets_wordpairSim/'+file.split('/')[-1].split('_')[-1], sep=';', names = column_names)
    
    r = pd.merge(pdf, df, on=['node1', 'node2'])
    score_human = r['simHuman']
    algo_score = r['sim']
    
    norm1 = [(float(i)-min(score_human))/(max(score_human )-min(score_human )) for i in score_human ] # normalize score in 0-1
    # calculate Pearson's correlation
    corr, _ = pearsonr(norm1, algo_score)
    pearson_ls.append(corr) 
 
    # calculate spearman's correlation
    corr, _ = spearmanr(norm1, algo_score)
    spearman_ls.append(corr) 

result_dict['Leacock : spearman'] = spearman_ls
result_dict['Leacock : Pearson'] = pearson_ls

denom =np.add (spearman_ls , pearson_ls)
ls = np.multiply(spearman_ls ,pearson_ls )
ls = [x * 2 for x in ls]
ls = ls/denom
#result_dict['Leacock : harmonic mean'] =  ls

In [50]:
df = pd.DataFrame(result_dict).T
df.columns = columns
df.style.highlight_max(color = 'lightgreen', axis = 0)

,mc30,ws353,verb143,rg65,YP130,simlex999,rw,wordsimSim
SP : spearman,0.509738,0.283972,0.196846,0.657737,0.473691,0.366731,0.198774,0.496549
SP : Pearson,0.489521,0.254977,0.133855,0.655934,0.467584,0.379607,0.189695,0.487518
Leacock : spearman,-0.383200,-0.114225,0.035563,-0.266795,-0.023329,-0.262912,-0.214483,-0.131478
Leacock : Pearson,-0.300321,-0.101877,-0.006449,-0.340009,0.017026,-0.263540,-0.225336,-0.128977


In [35]:
df_sematch = pd.read_csv('sematch_KRbased_NEWdatasets.csv')
df_sematch = df_sematch.set_index('Unnamed: 0')
df_final = round( pd.concat([df, df_sematch]),2)
df_final.style.highlight_max(color = 'lightgreen', axis = 0)

,mc30,verb143,mturk287,rg65,yp130,mturk771,wordsim353rel,simlex999,rw,men,wordsim353sim,simverb3500
SP : spearman,0.510000,0.480000,0.100000,0.640000,0.470000,0.280000,0.030000,0.170000,0.200000,0.260000,0.500000,0.220000
SP : Pearson,0.470000,0.480000,0.100000,0.610000,0.480000,0.310000,0.010000,0.230000,0.190000,0.260000,0.490000,0.220000
Leacock : spearman,-0.400000,-0.010000,0.050000,-0.230000,-0.020000,-0.100000,-0.090000,-0.200000,-0.210000,-0.020000,-0.140000,-0.040000
Leacock : Pearson,-0.300000,0.030000,0.020000,-0.310000,0.020000,-0.120000,-0.100000,-0.260000,-0.220000,-0.050000,-0.130000,-0.060000
wup : spearman,0.750000,0.070000,0.330000,0.760000,0.050000,0.440000,-0.010000,0.120000,-0.030000,0.330000,0.620000,0.000000
wup : Pearson,0.780000,0.100000,0.390000,0.790000,0.080000,0.410000,-0.010000,-0.010000,-0.030000,0.310000,0.580000,0.030000
path : spearman,0.720000,0.060000,0.330000,0.780000,0.050000,0.480000,-0.030000,0.130000,-0.040000,0.330000,0.600000,0.000000
path : Pearson,0.750000,0.180000,0.440000,0.780000,0.170000,0.420000,0.080000,0.190000,0.030000,0.370000,0.590000,0.130000
lch : spearman,0.720000,0.060000,0.330000,0.780000,0.050000,0.480000,-0.030000,0.130000,-0.040000,0.330000,0.600000,0.000000
lch : Pearson,0.780000,0.110000,0.390000,0.840000,0.090000,0.480000,0.010000,0.020000,-0.020000,0.320000,0.610000,0.040000


In [53]:
df_sematch = pd.read_csv('sematch_KRbased_result.csv')
df_sematch = df_sematch.set_index('Unnamed: 0')
df_final  = round( pd.concat([df, df_sematch]),2)
df_final.style.highlight_max(color = 'lightgreen', axis = 0)


,mc30,ws353,verb143,rg65,YP130,simlex999,rw,wordsimSim
SP : spearman,0.510000,0.280000,0.200000,0.660000,0.470000,0.370000,0.200000,0.500000
SP : Pearson,0.490000,0.250000,0.130000,0.660000,0.470000,0.380000,0.190000,0.490000
Leacock : spearman,-0.380000,-0.110000,0.040000,-0.270000,-0.020000,-0.260000,-0.210000,-0.130000
Leacock : Pearson,-0.300000,-0.100000,-0.010000,-0.340000,0.020000,-0.260000,-0.230000,-0.130000
wup : spearman,0.750000,0.350000,0.050000,0.540000,0.050000,0.550000,-0.030000,0.620000
wup : Pearson,0.780000,0.310000,-0.010000,0.520000,0.080000,0.540000,-0.030000,0.580000
path : spearman,0.720000,0.310000,0.060000,0.550000,0.050000,0.620000,-0.040000,0.600000
path : Pearson,0.750000,0.380000,0.020000,0.680000,0.170000,0.580000,0.030000,0.590000
lch : spearman,0.720000,0.310000,0.060000,0.550000,0.050000,0.620000,-0.040000,0.600000
lch : Pearson,0.780000,0.350000,0.010000,0.560000,0.090000,0.630000,-0.020000,0.610000


In [5]:
pd.read_csv('sem_Evaluation_KR.csv').style.highlight_max(color = 'lightgreen', axis = 0)

,Unnamed: 0,mc30,ws353,verb143,rg65,YP130,simlex999,rw,wordsimSim
0,SP : spearman,0.510000,0.280000,0.200000,0.680000,0.470000,0.370000,0.200000,0.500000
1,SP : Pearson,0.490000,0.250000,0.130000,0.660000,0.470000,0.380000,0.190000,0.490000
2,Leacock : spearman,-0.380000,-0.110000,0.040000,-0.270000,-0.020000,-0.260000,-0.210000,-0.130000
3,Leacock : Pearson,-0.300000,-0.100000,-0.010000,-0.340000,0.020000,-0.260000,-0.230000,-0.130000
4,wup : spearman,0.750000,0.350000,0.050000,0.540000,0.050000,0.550000,-0.030000,0.620000
5,wup : Pearson,0.780000,0.310000,-0.010000,0.520000,0.080000,0.540000,-0.030000,0.580000
6,path : spearman,0.720000,0.310000,0.060000,0.550000,0.050000,0.620000,-0.040000,0.600000
7,path : Pearson,0.750000,0.380000,0.020000,0.680000,0.170000,0.580000,0.030000,0.590000
8,lch : spearman,0.720000,0.310000,0.060000,0.550000,0.050000,0.620000,-0.040000,0.600000
9,lch : Pearson,0.780000,0.350000,0.010000,0.560000,0.090000,0.630000,-0.020000,0.610000


In [7]:
pd.read_csv('sem_Evaluation_IC.csv').style.highlight_max(color = 'lightgreen', axis = 0)

,Unnamed: 0,mc30,ws353,verb143,rg65,YP130,simlex999,rw,wordsimSim
0,SP : spearman,0.510000,0.280000,0.200000,0.680000,0.470000,0.370000,0.200000,0.500000
1,SP : Pearson,0.490000,0.250000,0.130000,0.660000,0.470000,0.380000,0.190000,0.490000
2,Leacock : spearman,-0.380000,-0.110000,0.040000,-0.270000,-0.020000,-0.260000,-0.210000,-0.130000
3,Leacock : Pearson,-0.300000,-0.100000,-0.010000,-0.340000,0.020000,-0.260000,-0.230000,-0.130000
4,wup : spearman,0.750000,0.350000,0.050000,0.540000,0.050000,0.550000,-0.030000,0.620000
5,wup : Pearson,0.780000,0.310000,-0.010000,0.520000,0.080000,0.540000,-0.030000,0.580000
6,path : spearman,0.720000,0.310000,0.060000,0.550000,0.050000,0.620000,-0.040000,0.600000
7,path : Pearson,0.750000,0.380000,0.020000,0.680000,0.170000,0.580000,0.030000,0.590000
8,lch : spearman,0.720000,0.310000,0.060000,0.550000,0.050000,0.620000,-0.040000,0.600000
9,lch : Pearson,0.780000,0.350000,0.010000,0.560000,0.090000,0.630000,-0.020000,0.610000


In [1]:
df_final.to_csv('sem_Evaluation_KR.csv')

NameError: name 'df_final' is not defined

In [55]:
df_sematch = pd.read_csv('sematch_ICbased_result.csv')
df_sematch = df_sematch.set_index('Unnamed: 0')
df_finalIC  = round( pd.concat([df, df_sematch]),2)
df_finalIC.style.highlight_max(color = 'lightgreen', axis = 0)

df_final.to_csv('sem_Evaluation_IC.csv')

In [45]:
df_final = np.round(df_final,3)
df_final['mean'] = round(df_final.mean(axis=1),2)

#df_final.T
df_final ['Type'] = ['KR', 'KR','KR', 'KR',  'KR', 'KR', 'KR','KR', 
                     'KR', 'KR','KR', 'KR']





ValueError: Length of values (12) does not match length of index (10)